In [10]:
    import fiona
    import os,os.path
    from shapely.geometry import shape,mapping
    from shapely.ops import transform
    from functools import partial
    import pyproj
    from fiona.crs import from_epsg
    import osmnx as ox
    import matplotlib.pyplot as plt
    import geopandas as gpd
    import glob
    import json
    import numpy as np
    import requests as rq
    import collections
    import logging
    import time
    import dateutil.parser
    import os
    from tqdm import tqdm
    %matplotlib inline

In [11]:
root= '/datasets_1/sagarj/BellLabs/CitySV_Data/city_centers_sampling/'

In [12]:
# Create a custom logger
logger = logging.getLogger(__name__)

# Create handlers
f_handler = logging.FileHandler('Mapillary.log')
f_handler.setLevel(logging.DEBUG)

# Create formatters and add it to handlers
f_format = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
f_handler.setFormatter(f_format)

# Add handlers to the logger
logger.addHandler(f_handler)

In [13]:
mapillary_Image_ID = 'https://a.mapillary.com/v3/images?client_id=bkJxVGFaQkdEcVY0RnVseWduREE0aTowZWMzMmY1Y2M3MjQ4ZTgx&closeto='
distanceFrag = '&radius=150'
mapillary_Image_url = 'https://images.mapillary.com/' #<image_key>/thumb-640.jpg

In [14]:
%store -r gdf

In [15]:
gdf.head(n=2)

,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,display_name,class,type,importance,centroid
0,"POLYGON ((-0.51038 51.46809, -0.51036 51.46795...",51.691874,51.286760,0.334016,-0.510375,256874343,relation,65606,51.507322,-0.127647,"London, Greater London, England, United Kingdom",boundary,administrative,0.940783,POINT (-0.10941 51.50050)
1,"POLYGON ((-87.94010 42.00093, -87.94003 41.998...",42.023040,41.644531,-87.524081,-87.940101,309792632,relation,122604,41.875562,-87.624421,"Chicago, Cook County, Illinois, United States",boundary,administrative,0.961530,POINT (-87.68569 41.83806)


In [16]:
# cities = os.listdir(root)
# print(cities)

['Helsinki, Helsinki sub-region, Uusimaa, Southern Finland, Mainland Finland, Finland', 'Madrid, Área metropolitana de Madrid y Corredor del Henares, Community of Madrid, 28001, Spain', 'Berlin, Germany', 'Moscow, Central Federal District, Russia', 'London, Greater London, England, United Kingdom', 'Munich, Bavaria, Germany', 'Zurich, District Zurich, Zurich, Switzerland', 'Paris, Ile-de-France, Metropolitan France, France', 'Vienna, Austria', 'Stockholms kommun, Stockholm County, Sweden', 'Manhattan, New York County, New York, United States', 'San Francisco, California, United States', 'Chicago, Cook County, Illinois, United States', 'Los Angeles, Los Angeles County, California, United States', 'Toronto, Golden Horseshoe, Ontario, Canada', 'Hamburg, Germany', 'Dusseldorf, North Rhine-Westphalia, Germany', 'Tokyo, Japan', 'Houston, Harris County, Texas, United States', 'Frankfurt, Hesse, Germany', 'New York, United States']


In [18]:
cities = ['Hamburg, Germany']

In [19]:
for c in tqdm(cities):
    cityRoot = root +'/'+ c
    city = c.split(',')[0].strip()
    data = gpd.read_file(cityRoot+'/shapeFiles/samples.shp')
    print("Total Sample points ",len(data) , c)
    count = 1
    for point in tqdm(data['geometry']):
        imagesMeta = []
        name= city + str(count) + '.json'
        loc= str(point.x)+','+str(point.y)
    #     print(loc)
        finalUrl = mapillary_Image_ID + loc + distanceFrag
        try:
            response = rq.get(finalUrl,timeout=10)
            dictionary = json.loads(response.text)
            imagesMeta = dictionary["features"]
            logger.debug('Working with %d image at location %s'%(count,loc))
        except:
            logger.error('Request Timed out')
    #     print(imagesMeta)
        logger.debug('Found %d Images in this file'%(count))
    #     print(imagesMeta)
        json.dump(imagesMeta,open(cityRoot+ '/' + 'pointsJson/' + name,'w'))
        count+=1
    print(os.listdir(root +'/'+ c))

  0%|          | 0/1999 [00:00<?, ?it/s]

Total Sample points  1999 Hamburg, Germany



  7%|▋         | 149/1999 [06:00<46:38,  1.51s/it]


 15%|█▍        | 296/1999 [13:01<32:02,  1.13s/it]


 22%|██▏       | 443/1999 [20:35<42:01,  1.62s/it]


...

 80%|███████▉  | 1598/1999 [1:31:39<10:29,  1.57s/it]


 88%|████████▊ | 1764/1999 [1:40:30<17:37,  4.50s/it]


 95%|█████████▌| 1907/1999 [1:46:53<00:39,  2.35it/s]


100%|██████████| 1/1 [1:49:55<00:00, 6595.39s/it]

['shapeFiles', 'pointsJson', 'Mapillary_images']
